In [30]:
%load_ext watermark
%watermark -a "Dhaval Antala" -vmp langchain,openai,pandas

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Author: Dhaval Antala

Python implementation: CPython
Python version       : 3.10.0
IPython version      : 8.25.0

langchain: 0.2.6
openai   : 1.32.0
pandas   : 1.5.3

Compiler    : Clang 12.0.0 
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit



In [31]:
import os
import openai
import warnings

warnings.filterwarnings("ignore")

In [32]:
# get your openai api key from https://platform.openai.com/account/api-keys 🔑
from getpass import getpass

OPENAI_API_KEY = getpass()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

In [33]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.llms import OpenAI

In [34]:
import pandas as pd
url = "https://raw.githubusercontent.com/sudarshan-koirala/Logistic-Regression-for-Titanic-Dataset/master/Train_Titanic.csv"
df = pd.read_csv(url)
print(df.shape)
df.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Single Dataframe Example.
- Agent can interact with single dataframe agent.

In [35]:
agent = create_pandas_dataframe_agent(OpenAI(temperature=0), df, verbose=True, allow_dangerous_code=True)

In [36]:
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows in the dataframe
Action: python_repl_ast
Action Input: len(df)891891 is the number of rows in the dataframe
Final Answer: 891

> Finished chain.


'891'

In [37]:
agent.run("how many people have more than 3 siblings")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include rows where the SibSp column is greater than 3, and then count the number of rows.
Action: python_repl_ast
Action Input: df[df['SibSp'] > 3].count()PassengerId    30
Survived       30
Pclass         30
Name           30
Sex            30
Age            23
SibSp          30
Parch          30
Ticket         30
Fare           30
Cabin           0
Embarked       30
dtype: int6430 rows have more than 3 siblings, but the Age and Cabin columns have missing values.
Final Answer: 30 people have more than 3 siblings.

> Finished chain.


'30 people have more than 3 siblings.'

### 📄📄 Multi DataFrame Example

- Agent can also interact with multiple dataframes passed in as a list.

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [39]:
df1 = df.copy()
df1["Age"] = df1["Age"].fillna(df1["Age"].mean())

In [41]:
agent = create_pandas_dataframe_agent(OpenAI(temperature=0), [df, df1], verbose=True, allow_dangerous_code=True)
agent.run("how many rows in the age column are different?")



> Entering new AgentExecutor chain...
Thought: I need to compare the age column in both dataframes and count the number of rows that are different.
Action: python_repl_ast
Action Input: df1['Age'].compare(df2['Age'])     self      other
5     NaN  29.699118
17    NaN  29.699118
19    NaN  29.699118
26    NaN  29.699118
28    NaN  29.699118
..    ...        ...
859   NaN  29.699118
863   NaN  29.699118
868   NaN  29.699118
878   NaN  29.699118
888   NaN  29.699118

[177 rows x 2 columns] I need to count the number of rows where the values in the self column are not equal to the values in the other column.
Action: python_repl_ast
Action Input: df1['Age'].compare(df2['Age']).notnull().sum()self       0
other    177
dtype: int64 I now know the final answer.
Final Answer: 177

> Finished chain.


'177'

In [42]:
df2 = df1.copy()
df2["Age_multiplied"] = df1["Age"] * 2
df2.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_multiplied
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,44.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,76.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,52.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,70.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,70.0


In [44]:
agent = create_pandas_dataframe_agent(OpenAI(temperature=0), [df, df1, df2], verbose=True, allow_dangerous_code=True)
agent.run("Are the number of columns same in all the dataframes?")



> Entering new AgentExecutor chain...
Thought: I should check the number of columns in each dataframe
Action: python_repl_ast
Action Input: len(df1.columns)1212 columns in df1
Action: python_repl_ast
Action Input: len(df2.columns)1212 columns in df2
Action: python_repl_ast
Action Input: len(df3.columns)1313 columns in df3
Final Answer: No, the number of columns is not the same in all the dataframes.

> Finished chain.


'No, the number of columns is not the same in all the dataframes.'